In [1]:
import numpy as np
import tensorflow as tf
import librosa
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,Add,Conv1D,MaxPooling1D
from keras.utils import np_utils
from IPython.display import Audio
from scipy.io import wavfile
import pandas as pd
import numpy as np
from IPython.display import Audio
from scipy.io import wavfile
import IPython.display as ipd

Using TensorFlow backend.


In [2]:
!git clone https://github.com/sanath493/dl-as1-audio.git

Cloning into 'dl-as1-audio'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [0]:
from zipfile import ZipFile
file_name = "dl-as1-audio/data.zip"
with ZipFile(file_name , 'r') as zip:
  zip.extractall()

In [0]:
s, sr=librosa.load('train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
S=np.transpose(S)
S=np.abs(S)
 
sn, sr=librosa.load('train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)
X=np.transpose(X)
X=np.abs(X)

sl, sr=librosa.load('test_x_01.wav', sr=None)
Y=librosa.stft(sl, n_fft=1024, hop_length=512)
Y=np.transpose(Y)
Y=np.abs(Y)


sa, sr=librosa.load('test_x_02.wav', sr=None)
Z=librosa.stft(sa, n_fft=1024, hop_length=512)
Z=np.transpose(Z)
Z=np.abs(Z)


In [0]:
nx,ny=Y.shape
px,py=Z.shape

In [0]:
S=S.reshape(2459,513)
X=X.reshape(2459,513,1)
Y=Y.reshape(nx,ny,1)
Z=Z.reshape(px,py,1)

In [7]:
model =Sequential()
model.add(Conv1D(filters=16, kernel_size=(5) ,strides=2,input_shape=(513,1),kernel_initializer= 'glorot_normal',activation= 'relu'))
model.add(MaxPooling1D(2))
model.add(Conv1D(filters=16, kernel_size=(5) ,strides=2,kernel_initializer= 'uniform',activation= 'relu'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(513,activation=tf.nn.relu))
model.compile(optimizer='adam',loss='mse', metrics=['accuracy'])
model.summary()
model.fit(X,S,epochs = 100,batch_size=64)







Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 255, 16)           96        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 127, 16)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 62, 16)            1296      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 31, 16)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 496)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 513)               254961    
Total params: 256,353
Trainable params: 256,353
Non-trainable params: 0
__________________________________________

In [0]:
y_pred1_train = model.predict(X)
K1=np.transpose(y_pred1_train)
sn, sr=librosa.load('train_dirty_male.wav', sr=None)
X1=librosa.stft(sn, n_fft=1024, hop_length=512)
X2=np.abs(X1)
z1=(X1/X2)
z1.shape
o1=np.multiply(z1,K1)
Op_test=librosa.istft(o1,hop_length=512,length=len(s))

In [9]:
nume=pow(s,2)
deno=[]
for i in range(len(Op_test)):
  deno.append(s[i]-Op_test[i])
deno=np.asarray(deno)
nume=nume.sum()
deno=pow(deno,2)
deno=deno.sum()
out=(nume/deno)
snr=10*np.log10(out)
snr

14.026930332183838

In [0]:
y_pred1 = model.predict(Y)
K=np.transpose(y_pred1)
sp, sr=librosa.load('test_x_01.wav', sr=None)
Z1=librosa.stft(sp, n_fft=1024, hop_length=512)
Z2=np.abs(Z1)
z=(Z1/Z2)
z.shape
o=np.multiply(z,K)
Op=librosa.istft(o,hop_length=512)

In [18]:
z.shape

(513, 142)

In [0]:
librosa.output.write_wav('test_s_01_recons1_1dcnn.wav', Op, sr)

In [12]:
ipd.Audio('test_x_01.wav')

In [13]:
ipd.Audio('test_s_01_recons1_1dcnn.wav')

In [0]:
y_pred2 = model.predict(Z)
K1=np.transpose(y_pred2)
sa, sr=librosa.load('test_x_02.wav', sr=None)
Y1=librosa.stft(sa, n_fft=1024, hop_length=512)
Y2=np.abs(Y1)
C=(Y1/Y2)
C.shape
L=np.multiply(C,K1)
Op1=librosa.istft(L,hop_length=512)

In [0]:
librosa.output.write_wav('test_s_01_recons2_1dcnn.wav', Op1, sr)

In [16]:
ipd.Audio('test_x_02.wav')

In [17]:
ipd.Audio('test_s_01_recons2_1dcnn.wav')